In [ ]:
import fitz  # PyMuPDF
import re, json

# Upload your PDF manually in Colab sidebar (Files > Upload)
pdf_path = "../data/raw/OSH.pdf"  # Change name if needed

# ======================================
# 📄 STEP 2: Extract Text from PDF
# ======================================

doc = fitz.open(pdf_path)
full_text = ""

for page_num, page in enumerate(doc, start=1):
    text = page.get_text("text")
    full_text += text + "\n"

print("✅ Extraction complete. Total characters:", len(full_text))
print(full_text[:1000])  # preview first 1000 chars

# ======================================
# 🧹 STEP 3: Clean Text
# ======================================

clean_text = re.sub(r'\n+', '\n', full_text)  # remove multiple newlines
clean_text = re.sub(r'Page\s*\d+\s*of\s*\d+', '', clean_text)  # remove page numbers
clean_text = re.sub(r'THE GAZETTE OF INDIA', '', clean_text, flags=re.IGNORECASE)
clean_text = re.sub(r'Ministry of Labour.*?\n', '', clean_text, flags=re.IGNORECASE)  # remove ministry headers

: 

In [ ]:
chapters = re.split(r'(?=CHAPTER\s+[IVXLC]+\s+—)', clean_text)

print(f"✅ Found {len(chapters)} chapters.")
for i, ch in enumerate(chapters[:3]):  # show first 3
    print(f"\n--- Chapter {i+1} ---\n{ch[:400]}")

# ======================================
# 🔖 STEP 5: Split Chapters into SECTIONS
# ======================================

sections = re.split(r'(?=\n\d+\.\s)', clean_text)
print(f"✅ Found approximately {len(sections)} sections.")

for i, sec in enumerate(sections[:5]):
    print(f"\n--- Section {i+1} ---\n{sec[:400]}")

In [ ]:
data = [{"id": i+1, "text": sec.strip()}
         for i, sec in enumerate(sections)
         if len(sec.strip()) > 50]

# Reassign sequential IDs
for new_id, item in enumerate(data, start=1):
    item["id"] = new_id


output_file = "osh_sections.json"

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"✅ Saved {len(data)} sections into {output_file}!")

# ======================================
# 🧾 STEP 7: Download the JSON
# ======================================

# from google.colab import files
# files.download(output_file)